In [1]:
import numpy as np
import pandas as pd
from numpy import nan
import pickle
pd.set_option('display.max_rows', 200)

## Data Cleaning

### 1. load data

In [2]:
comprehensive = pd.read_stata('../Data/data/Comprehensive-Sample.dta')

In [16]:
comprehensive.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17920 entries, 0 to 17919
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        17920 non-null  float32
 1   numcode                     17920 non-null  int16  
 2   oilreserves_full            14307 non-null  float32
 3   oilreserves                 12519 non-null  float32
 4   oilreserves_public          9950 non-null   float32
 5   newdiscovery_aspo           12354 non-null  float32
 6   aspo                        4736 non-null   float32
 7   wildcat                     12354 non-null  float32
 8   endowment                   14160 non-null  float32
 9   pop_maddison                10441 non-null  float64
 10  ecgrowth                    9341 non-null   float32
 11  efrac                       390 non-null    float32
 12  lfrac                       390 non-null    float32
 13  rfrac                       390

In [5]:
aspo = pd.read_stata('../Data/data/ASPO-Sample.dta')

### 2. function to do transformation

In [48]:
# Create a sample dataframe
df = pd.DataFrame({"country": np.random.choice(["A", "B", "C"], size=20),
                   "year": np.arange(20),
                   "lognum1": np.random.randint(1, 4, size=20),
                   "num2": np.random.randint(20000, 90000, size=20),
                   "num3": np.random.randint(20000, 90000, size=20),
                   "num4": np.random.randint(20000, 90000, size=20)})

In [49]:
df = df.sort_values(by=['country','year'], ascending=True)

In [50]:
df

,country,year,lognum1,num2,num3,num4
0,A,0,1,53289,84438,33081
5,A,5,3,89634,28144,49934
6,A,6,2,89617,58367,30182
10,A,10,1,76584,45595,79870
14,A,14,3,61607,78038,77474
18,A,18,3,89319,79395,24936
19,A,19,2,82841,75240,33007
1,B,1,2,72677,59579,65416
2,B,2,1,79880,24483,52615
3,B,3,2,32197,33476,87753


In [3]:
def transform_df(df, groupby_cols, log_cols, diff_cols, pct_change_cols, binarize_cols):
    
    new_df = df.copy()
    new_cols = []

    for i, col_name in enumerate(log_cols):
        new_col_name = col_name[3:] # to get rid of 'log' in the column name
        new_df[new_col_name] = np.exp(new_df[col_name])
        log_cols[i] = new_col_name

    all_cols = log_cols + pct_change_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)

        if col_name in binarize_cols:
            binarize_name = col_name + '_binarize'
            new_df[binarize_name] = np.NAN
            new_df.loc[new_df[diff_name] > 0.1, binarize_name] = 1
            new_df.loc[new_df[diff_name] <= 0.1, binarize_name] = 0
            new_df.loc[pd.isnull(new_df[diff_name]), binarize_name] = np.nan
            new_cols.append(binarize_name)

    for col_name in diff_cols:

        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df[col_name] - new_df.groupby(groupby_cols)[col_name].shift(1)
        new_cols.append(diff_name)
            
    return new_df[new_cols]

In [56]:
transform_df(df, ['country'], ['lognum1'], ['num4'], ['num2','num3'], ['num3'])

,num1_diff,num2_diff,num3_diff,num3_binarize,num4_diff
0,NaN,NaN,NaN,NaN,NaN
5,6.389056,0.682036,-0.666690,0.0,16853.0
6,-0.632121,-0.000190,1.073870,1.0,-19752.0
10,-0.632121,-0.145430,-0.218822,0.0,49688.0
14,6.389056,-0.195563,0.711547,1.0,-2396.0
18,0.000000,0.449819,0.017389,0.0,-52538.0
19,-0.632121,-0.072527,-0.052333,0.0,8071.0
1,NaN,NaN,NaN,NaN,NaN
2,-0.632121,0.099110,-0.589067,0.0,-12801.0
3,1.718282,-0.596933,0.367316,1.0,35138.0


--

### 3. comprehensive - cleaning

In [45]:
v_lst = ['onset2COWCS',
         'onsetUCS',
         'coup',
         'periregular',
         'milexp_pergdpSIPRI',
         #'logmilexgdpSIPRI',
         'numcode',
         'year',
         
         #'logGDP_M',
         'ecgrowth',
         'pop_maddison',
         #'logpop_M',
         'logpopdens', ####
         'democracy',
         'logmountain',
         'ethnic_fractionalization',
         'religion_fractionalization',
         'language_fractionalization',
         'leg_british',
         'no_transition',
         'wildcat',
         
         'valoilres', # value of oil reserves
         #'logvaloilres',
         'valoilres_public',
         #'logvaloilres_public',
         'oilpop',
         #'logoilres',
         'valoilres_impute',
         #'logvaloilres_impute',
         'oilpop_impute',
         #'logoilres_impute',
         'out_regdisaster']
         #'logoutreg'

In [62]:
# variables that need transformation
# transform_lst = ['logmilexgdpSIPRI', # defense burden
#                 'logpop_M',
#                 'logpopdens',
#                 'democracy', # democracy index
#                 'wildcat', # wildcat drilling 
#                 'logoutreg', # out of region disaster 
#                 'logvaloilres',
#                 'logvaloilres_public',
#                 'logoilres',
#                 'logvaloilres_impute',
#                 'logoilres_impute'
#                 ]

transform_lst = ['milexp_pergdpSIPRI',
                'pop_maddison',
                'logpopdens',
                'democracy',
                'wildcat',
                'out_regdisaster',
                'valoilres', # value of oil reserves
                'valoilres_public',
                'oilpop',
                'valoilres_impute',
                'oilpop_impute']

In [4]:
# variables that need to take exp and then compute pct change
log_lst = ['logpopdens']

# variables that need to take diff directly
diff_lst = ['milexp_pergdpSIPRI']

# variables that need to take pct change directly
pct_change_lst = ['pop_maddison', 
                'democracy',
                'wildcat',
                'out_regdisaster',
                'valoilres', # value of oil reserves
                'valoilres_public', # value of oil reserves from public data
                'oilpop', # oil reserves per capita in million barrels per 1000 persons
                'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
                'oilpop_impute'] # oil reserves per capita - imputed

# a subset of pct_change_lst
# variables that need to be binarized
binarize_lst = ['valoilres', # value of oil reserves
              'valoilres_public', # value of oil reserves from public data
              'oilpop', # oil reserves per capita in million barrels per 1000 persons
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

# variables that don't need transformation
untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [5]:
comprehensive = comprehensive.replace(0, 0.1**5)

In [6]:
comprehensive = comprehensive.sort_values(by=['numcode','year'], ascending=True)

In [7]:
new_df = transform_df(df = comprehensive,
                      groupby_cols = ['numcode'], 
                      log_cols = log_lst, 
                      diff_cols = diff_lst, 
                      pct_change_cols = pct_change_lst, 
                      binarize_cols = binarize_lst)
                      

In [10]:
new_df

,popdens_diff,pop_maddison_diff,democracy_diff,wildcat_diff,out_regdisaster_diff,valoilres_diff,valoilres_binarize,valoilres_public_diff,valoilres_public_binarize,oilpop_diff,oilpop_binarize,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize,milexp_pergdpSIPRI_diff
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.000000,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0.000000,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.000000,NaN,3.566128e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,0.000149,0.015008,0.000000,0.0,8.259656e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17916,0.000139,0.013990,0.000000,0.0,1.678100e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17917,0.000154,0.015553,0.000000,0.0,-8.735424e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17918,0.000166,0.016760,0.000000,0.0,8.096983e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [11]:
comprehensive_new = pd.concat([comprehensive[untransformed_lst], new_df], axis = 1)

In [12]:
comprehensive_new.shape

(17920, 29)

In [13]:
comprehensive_new.columns

Index(['onset2COWCS', 'onsetUCS', 'coup', 'periregular', 'numcode', 'year',
       'ecgrowth', 'logmountain', 'ethnic_fractionalization',
       'religion_fractionalization', 'language_fractionalization',
       'leg_british', 'no_transition', 'popdens_diff', 'pop_maddison_diff',
       'democracy_diff', 'wildcat_diff', 'out_regdisaster_diff',
       'valoilres_diff', 'valoilres_binarize', 'valoilres_public_diff',
       'valoilres_public_binarize', 'oilpop_diff', 'oilpop_binarize',
       'valoilres_impute_diff', 'valoilres_impute_binarize',
       'oilpop_impute_diff', 'oilpop_impute_binarize',
       'milexp_pergdpSIPRI_diff'],
      dtype='object')

In [16]:
# potential selection of variables
comprehensive_new[['onset2COWCS',
                   'valoilres_binarize',
                   'ecgrowth',
                   'pop_maddison_diff',
                   'popdens_diff',
                   'democracy_diff',
                   'logmountain',
                   'ethnic_fractionalization',
                   'religion_fractionalization',
                   'language_fractionalization',
                   'leg_british',
                   'numcode',
                   'year']].dropna()

,onset2COWCS,valoilres_binarize,ecgrowth,pop_maddison_diff,popdens_diff,democracy_diff,logmountain,ethnic_fractionalization,religion_fractionalization,language_fractionalization,leg_british,numcode,year
195,0.00001,0.0,0.021566,0.030159,0.000297,0.000000,0.027537,0.003394,0.000091,0.004427,0.00001,12,1964.0
196,0.00001,0.0,0.035326,0.030159,0.000297,-0.500000,0.027537,0.003394,0.000091,0.004427,0.00001,12,1965.0
197,0.00001,0.0,-0.077295,0.031434,0.000310,0.000000,0.027537,0.003394,0.000091,0.004427,0.00001,12,1966.0
198,0.00001,0.0,0.057393,0.034148,0.000336,0.000000,0.027537,0.003394,0.000091,0.004427,0.00001,12,1967.0
199,0.00001,0.0,0.084017,0.030231,0.000298,0.000000,0.027537,0.003394,0.000091,0.004427,0.00001,12,1968.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17912,0.00001,0.0,0.025446,0.022981,0.000227,0.363636,-0.016094,0.007808,0.007359,0.008734,0.01000,894,2001.0
17913,0.00001,0.0,0.014076,0.018515,0.000184,0.000000,-0.016094,0.007808,0.007359,0.008734,0.01000,894,2002.0
17914,0.00001,0.0,0.034933,0.015692,0.000156,0.000000,-0.016094,0.007808,0.007359,0.008734,0.01000,894,2003.0
17915,0.00001,0.0,0.038210,0.015008,0.000149,0.000000,-0.016094,0.007808,0.007359,0.008734,0.01000,894,2004.0


In [32]:
comprehensive_new[['oilpop_impute_binarize','year']].groupby('oilpop_impute_binarize').count()

,year
oilpop_impute_binarize,
0.0,7527
1.0,398


In [72]:
comprehensive.to_pickle("../Data/data/comprehensive_new.pkl")  

In [33]:
comprehensive_new

,onset2COWCS,onsetUCS,coup,periregular,numcode,year,ecgrowth,logmountain,ethnic_fractionalization,religion_fractionalization,...,valoilres_binarize,valoilres_public_diff,valoilres_public_binarize,oilpop_diff,oilpop_binarize,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize,milexp_pergdpSIPRI_diff
0,0.00001,NaN,NaN,NaN,4,1929.0,NaN,0.041836,0.007693,0.002717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00001,NaN,NaN,0.00001,4,1930.0,NaN,0.041836,0.007693,0.002717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.00001,NaN,NaN,0.00001,4,1931.0,NaN,0.041836,0.007693,0.002717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.00001,NaN,NaN,0.00001,4,1932.0,NaN,0.041836,0.007693,0.002717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.00001,NaN,NaN,0.00001,4,1933.0,NaN,0.041836,0.007693,0.002717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,0.00001,0.00001,0.00001,NaN,894,2004.0,0.038210,-0.016094,0.007808,0.007359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17916,0.00001,0.00001,0.00001,NaN,894,2005.0,0.038433,-0.016094,0.007808,0.007359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17917,NaN,0.00001,0.00001,NaN,894,2006.0,0.045788,-0.016094,0.007808,0.007359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
17918,NaN,0.00001,0.00001,NaN,894,2007.0,0.045470,-0.016094,0.007808,0.007359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [48]:
comprehensive_new['valoilres_binarize'].describe()

count    7873.000000
mean        0.111266
std         0.314482
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: valoilres_binarize, dtype: float64

In [43]:
comprehensive['valoilres_impute'].describe()

count     7620.000000
mean       124.457787
std        849.155640
min          0.000010
25%          0.000010
50%          0.000010
75%          3.599900
max      21176.099609
Name: valoilres_impute, dtype: float64

In [47]:
(comprehensive_new['valoilres_impute_binarize'] == comprehensive_new['valoilres_binarize']).sum()

7650

In [ ]:
for col_name in comprehensive_new.columns:
    print(comprehensive_new[col_name].describe())

In [49]:
np.nan == np.nan

False

In [56]:
s = pd.Series([np.nan, 0.0001, 90, 91, 85])
s.pct_change()

0              NaN
1              NaN
2    899999.000000
3         0.011111
4        -0.065934
dtype: float64